In [14]:
import os
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import glob
from tqdm.auto import tqdm
import cv2

In [64]:
import os
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import glob
from PIL import Image
from matplotlib import pyplot as plt
%matplotlib inline
from tqdm import tqdm
import cv2

from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dropout,Flatten
from keras.layers.convolutional import Conv2D,Conv1D
from keras.layers.pooling import MaxPooling2D,AveragePooling2D,GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout,Flatten
from keras.applications import VGG16
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing import image

Using TensorFlow backend.


In [69]:
class Helper():
    
    def load_files(self,path,categories):
        images = []
        labels = []
        for category in tqdm(categories,total=len(categories)):
            file_path = os.path.join(path,category+'/'+category[0:3])
            #print(file_path)
            files = glob.glob(file_path+"*.jpg")
            for file in tqdm(files,total=len(files)):
                #print(file)
                img_array = cv2.imread(file,cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img_array,dsize=(IMG_SIZE,IMG_SIZE))
                img = img/255.0
                #img = np.array(img).reshape(-1,IMG_SIZE,IMG_SIZE,1)
                label = category[0:3]
                images.append(img)
                labels.append(label)
        image_array = np.array(images)
        label_array = np.array(labels)
        return image_array,label_array

In [70]:
cwd = os.getcwd()
training_path = os.path.join(cwd,"dataset/training_set")
IMG_SIZE = 80
categories = ["dogs","cats"]
helper = Helper()
images,labels = helper.load_files(training_path,categories)

 77%|███████████████████████████████████████████████████████████▏                 | 3074/4000 [00:17<00:06, 143.14it/s]


 40%|██████████████████████████████▌                                              | 1590/4000 [00:09<00:13, 184.41it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:48<00:00, 24.32s/it]


In [71]:
img_label = pd.DataFrame(data=[images,labels]).T
img_label.columns = ["image","label"]

In [72]:
img_label = img_label.sample(frac=1)

In [73]:
images = img_label["image"]
labels = img_label["label"]

In [81]:
images[0].shape

(8000, 80, 80)

In [77]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

labels = encoder.fit_transform(labels[0])

In [1]:
pickle.dump(encoder,open("encoder.pkl","wb"))

NameError: name 'pickle' is not defined

In [82]:
X = np.array(images[0]).reshape(-1, IMG_SIZE,IMG_SIZE,1)
y = np.array(labels)

In [83]:
model = Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(Conv2D(64,(3,3), activation = 'relu', input_shape = X.shape[1:]))
model.add(MaxPooling2D(pool_size = (2,2)))
# Add another:
model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [84]:
model.fit(X, y, epochs=25, batch_size=32, validation_split=0.25)

Train on 6000 samples, validate on 2000 samples
Epoch 1/25


5920/6000 [============================>.] - ETA: 10:38 - loss: 0.6869 - accuracy: 0.687 - ETA: 7:33 - loss: 0.7244 - accuracy: 0.703 - ETA: 5:56 - loss: 0.8065 - accuracy: 0.61 - ETA: 5:00 - loss: 0.7657 - accuracy: 0.62 - ETA: 4:29 - loss: 0.7479 - accuracy: 0.63 - ETA: 4:07 - loss: 0.7365 - accuracy: 0.65 - ETA: 3:48 - loss: 0.7289 - accuracy: 0.66 - ETA: 3:37 - loss: 0.7218 - accuracy: 0.67 - ETA: 3:28 - loss: 0.7157 - accuracy: 0.67 - ETA: 3:19 - loss: 0.7103 - accuracy: 0.67 - ETA: 3:14 - loss: 0.7063 - accuracy: 0.67 - ETA: 3:07 - loss: 0.7006 - accuracy: 0.67 - ETA: 3:01 - loss: 0.6975 - accuracy: 0.67 - ETA: 2:58 - loss: 0.6961 - accuracy: 0.66 - ETA: 2:54 - loss: 0.6944 - accuracy: 0.66 - ETA: 2:50 - loss: 0.6904 - accuracy: 0.66 - ETA: 2:47 - loss: 0.6860 - accuracy: 0.66 - ETA: 2:44 - loss: 0.6801 - accuracy: 0.67 - ETA: 2:41 - loss: 0.6802 - accuracy: 0.66 - ETA: 2:39 - loss: 0.6820 - accuracy: 0.66 - ETA: 2:37 - loss: 0.6789 - accuracy: 0.66 - ETA: 2:34 - loss: 0.6772 - a

5952/6000 [============================>.] - ETA: 2:46 - loss: 0.6223 - accuracy: 0.62 - ETA: 2:23 - loss: 0.6018 - accuracy: 0.64 - ETA: 2:25 - loss: 0.6400 - accuracy: 0.58 - ETA: 2:22 - loss: 0.6087 - accuracy: 0.64 - ETA: 2:22 - loss: 0.5903 - accuracy: 0.67 - ETA: 2:22 - loss: 0.5984 - accuracy: 0.66 - ETA: 2:22 - loss: 0.6052 - accuracy: 0.65 - ETA: 2:22 - loss: 0.6077 - accuracy: 0.64 - ETA: 2:18 - loss: 0.6114 - accuracy: 0.63 - ETA: 2:18 - loss: 0.6044 - accuracy: 0.64 - ETA: 2:19 - loss: 0.6085 - accuracy: 0.64 - ETA: 2:16 - loss: 0.6107 - accuracy: 0.65 - ETA: 2:16 - loss: 0.6085 - accuracy: 0.65 - ETA: 2:16 - loss: 0.6114 - accuracy: 0.64 - ETA: 2:14 - loss: 0.6060 - accuracy: 0.66 - ETA: 2:14 - loss: 0.6094 - accuracy: 0.65 - ETA: 2:13 - loss: 0.6047 - accuracy: 0.66 - ETA: 2:12 - loss: 0.6057 - accuracy: 0.66 - ETA: 2:11 - loss: 0.6008 - accuracy: 0.66 - ETA: 2:10 - loss: 0.5935 - accuracy: 0.67 - ETA: 2:09 - loss: 0.5916 - accuracy: 0.67 - ETA: 2:08 - loss: 0.5864 - accu

6000/6000 [==============================] - ETA: 0s - loss: 0.5629 - accuracy: 0.70 - 169s 28ms/step - loss: 0.5628 - accuracy: 0.7055 - val_loss: 0.7373 - val_accuracy: 0.5145
Epoch 3/25


5952/6000 [============================>.] - ETA: 2:26 - loss: 0.4609 - accuracy: 0.90 - ETA: 2:26 - loss: 0.4930 - accuracy: 0.84 - ETA: 2:26 - loss: 0.5190 - accuracy: 0.80 - ETA: 2:32 - loss: 0.5233 - accuracy: 0.78 - ETA: 2:33 - loss: 0.5399 - accuracy: 0.77 - ETA: 2:31 - loss: 0.5395 - accuracy: 0.77 - ETA: 2:35 - loss: 0.5455 - accuracy: 0.75 - ETA: 2:36 - loss: 0.5312 - accuracy: 0.75 - ETA: 2:36 - loss: 0.5408 - accuracy: 0.75 - ETA: 2:40 - loss: 0.5334 - accuracy: 0.75 - ETA: 2:40 - loss: 0.5220 - accuracy: 0.75 - ETA: 2:40 - loss: 0.5269 - accuracy: 0.75 - ETA: 2:40 - loss: 0.5184 - accuracy: 0.76 - ETA: 2:39 - loss: 0.5159 - accuracy: 0.76 - ETA: 2:39 - loss: 0.5124 - accuracy: 0.76 - ETA: 2:37 - loss: 0.5208 - accuracy: 0.75 - ETA: 2:35 - loss: 0.5284 - accuracy: 0.75 - ETA: 2:33 - loss: 0.5325 - accuracy: 0.74 - ETA: 2:33 - loss: 0.5315 - accuracy: 0.74 - ETA: 2:30 - loss: 0.5278 - accuracy: 0.75 - ETA: 2:28 - loss: 0.5320 - accuracy: 0.74 - ETA: 2:26 - loss: 0.5344 - accu

...

6000/6000 [==============================] - ETA: 15s - loss: 7.0576e-04 - accuracy: 1.000 - ETA: 14s - loss: 7.0537e-04 - accuracy: 1.000 - ETA: 13s - loss: 7.0655e-04 - accuracy: 1.000 - ETA: 12s - loss: 7.0441e-04 - accuracy: 1.000 - ETA: 11s - loss: 7.0414e-04 - accuracy: 1.000 - ETA: 10s - loss: 7.0372e-04 - accuracy: 1.000 - ETA: 9s - loss: 7.0422e-04 - accuracy: 1.000 - ETA: 8s - loss: 7.0233e-04 - accuracy: 1.00 - ETA: 7s - loss: 7.0088e-04 - accuracy: 1.00 - ETA: 6s - loss: 7.0130e-04 - accuracy: 1.00 - ETA: 5s - loss: 6.9969e-04 - accuracy: 1.00 - ETA: 5s - loss: 6.9781e-04 - accuracy: 1.00 - ETA: 4s - loss: 6.9757e-04 - accuracy: 1.00 - ETA: 3s - loss: 6.9664e-04 - accuracy: 1.00 - ETA: 2s - loss: 6.9712e-04 - accuracy: 1.00 - ETA: 1s - loss: 7.0184e-04 - accuracy: 1.00 - ETA: 0s - loss: 7.0536e-04 - accuracy: 1.00 - 187s 31ms/step - loss: 7.0359e-04 - accuracy: 1.0000 - val_loss: 3.0196 - val_accuracy: 0.6345
Epoch 23/25


5440/6000 [==========================>...] - ETA: 3:19 - loss: 4.2818e-04 - accuracy: 1.00 - ETA: 3:02 - loss: 5.4718e-04 - accuracy: 1.00 - ETA: 2:53 - loss: 4.9856e-04 - accuracy: 1.00 - ETA: 2:52 - loss: 5.8912e-04 - accuracy: 1.00 - ETA: 2:49 - loss: 5.3465e-04 - accuracy: 1.00 - ETA: 2:45 - loss: 4.8450e-04 - accuracy: 1.00 - ETA: 2:46 - loss: 4.5041e-04 - accuracy: 1.00 - ETA: 2:44 - loss: 4.3404e-04 - accuracy: 1.00 - ETA: 2:42 - loss: 4.4429e-04 - accuracy: 1.00 - ETA: 2:42 - loss: 4.2197e-04 - accuracy: 1.00 - ETA: 2:40 - loss: 4.2134e-04 - accuracy: 1.00 - ETA: 2:39 - loss: 4.2913e-04 - accuracy: 1.00 - ETA: 2:39 - loss: 4.2713e-04 - accuracy: 1.00 - ETA: 2:37 - loss: 4.4121e-04 - accuracy: 1.00 - ETA: 2:36 - loss: 4.4553e-04 - accuracy: 1.00 - ETA: 2:35 - loss: 4.5215e-04 - accuracy: 1.00 - ETA: 2:33 - loss: 4.6087e-04 - accuracy: 1.00 - ETA: 2:33 - loss: 4.5665e-04 - accuracy: 1.00 - ETA: 2:32 - loss: 4.5125e-04 - accuracy: 1.00 - ETA: 2:30 - loss: 4.7622e-04 - accuracy: 1.

5440/6000 [==========================>...] - ETA: 2:35 - loss: 7.6532e-04 - accuracy: 1.00 - ETA: 2:46 - loss: 6.2635e-04 - accuracy: 1.00 - ETA: 2:45 - loss: 6.4114e-04 - accuracy: 1.00 - ETA: 2:37 - loss: 5.3783e-04 - accuracy: 1.00 - ETA: 2:40 - loss: 5.3456e-04 - accuracy: 1.00 - ETA: 2:49 - loss: 5.3467e-04 - accuracy: 1.00 - ETA: 2:49 - loss: 5.0827e-04 - accuracy: 1.00 - ETA: 2:49 - loss: 6.1077e-04 - accuracy: 1.00 - ETA: 2:51 - loss: 5.9250e-04 - accuracy: 1.00 - ETA: 2:49 - loss: 5.7494e-04 - accuracy: 1.00 - ETA: 2:48 - loss: 5.6932e-04 - accuracy: 1.00 - ETA: 2:46 - loss: 6.0298e-04 - accuracy: 1.00 - ETA: 2:45 - loss: 6.0036e-04 - accuracy: 1.00 - ETA: 2:44 - loss: 6.0174e-04 - accuracy: 1.00 - ETA: 2:41 - loss: 6.0734e-04 - accuracy: 1.00 - ETA: 2:40 - loss: 6.0663e-04 - accuracy: 1.00 - ETA: 2:39 - loss: 6.0362e-04 - accuracy: 1.00 - ETA: 2:37 - loss: 5.9588e-04 - accuracy: 1.00 - ETA: 2:37 - loss: 6.2044e-04 - accuracy: 1.00 - ETA: 2:37 - loss: 6.4690e-04 - accuracy: 1.

6000/6000 [==============================] - ETA: 15s - loss: 4.7020e-04 - accuracy: 1.000 - ETA: 14s - loss: 4.6926e-04 - accuracy: 1.000 - ETA: 13s - loss: 4.6948e-04 - accuracy: 1.000 - ETA: 12s - loss: 4.6960e-04 - accuracy: 1.000 - ETA: 11s - loss: 4.6834e-04 - accuracy: 1.000 - ETA: 10s - loss: 4.6841e-04 - accuracy: 1.000 - ETA: 9s - loss: 4.6686e-04 - accuracy: 1.000 - ETA: 8s - loss: 4.6752e-04 - accuracy: 1.00 - ETA: 7s - loss: 4.6917e-04 - accuracy: 1.00 - ETA: 6s - loss: 4.7105e-04 - accuracy: 1.00 - ETA: 5s - loss: 4.6910e-04 - accuracy: 1.00 - ETA: 5s - loss: 4.6833e-04 - accuracy: 1.00 - ETA: 4s - loss: 4.6720e-04 - accuracy: 1.00 - ETA: 3s - loss: 4.6828e-04 - accuracy: 1.00 - ETA: 2s - loss: 4.6724e-04 - accuracy: 1.00 - ETA: 1s - loss: 4.6638e-04 - accuracy: 1.00 - ETA: 0s - loss: 4.6519e-04 - accuracy: 1.00 - 186s 31ms/step - loss: 4.6554e-04 - accuracy: 1.0000 - val_loss: 3.1347 - val_accuracy: 0.6370
Epoch 25/25


5440/6000 [==========================>...] - ETA: 2:46 - loss: 5.6570e-04 - accuracy: 1.00 - ETA: 2:43 - loss: 4.0186e-04 - accuracy: 1.00 - ETA: 2:34 - loss: 3.8501e-04 - accuracy: 1.00 - ETA: 2:32 - loss: 3.6875e-04 - accuracy: 1.00 - ETA: 2:35 - loss: 3.7259e-04 - accuracy: 1.00 - ETA: 2:32 - loss: 4.2455e-04 - accuracy: 1.00 - ETA: 2:29 - loss: 4.6926e-04 - accuracy: 1.00 - ETA: 2:29 - loss: 5.0426e-04 - accuracy: 1.00 - ETA: 2:28 - loss: 4.9745e-04 - accuracy: 1.00 - ETA: 2:26 - loss: 4.7999e-04 - accuracy: 1.00 - ETA: 2:26 - loss: 4.8193e-04 - accuracy: 1.00 - ETA: 2:26 - loss: 4.7397e-04 - accuracy: 1.00 - ETA: 2:24 - loss: 4.7310e-04 - accuracy: 1.00 - ETA: 2:23 - loss: 4.5231e-04 - accuracy: 1.00 - ETA: 2:22 - loss: 4.4910e-04 - accuracy: 1.00 - ETA: 2:21 - loss: 4.4837e-04 - accuracy: 1.00 - ETA: 2:21 - loss: 4.4631e-04 - accuracy: 1.00 - ETA: 2:20 - loss: 4.7255e-04 - accuracy: 1.00 - ETA: 2:19 - loss: 4.6703e-04 - accuracy: 1.00 - ETA: 2:18 - loss: 4.6628e-04 - accuracy: 1.

In [86]:
import pickle
pickle.dump(model,open("dogs_cats_classification_model.pkl","wb"))